In [1]:
import pandas as pd
import numpy as np
import os

from tqdm import tqdm
from unicodedata import normalize

from src.data import cleandata

In [2]:
tqdm.pandas()

path_interim = "../data/interim/pjud"
path_processed = "../data/processed/pjud"

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# DATA de TRIBUNALES OBTENIDAS DEL COT

In [3]:
# Analizo contra los datos extraidos en COT
df_tribunales = pd.read_feather(f"{path_processed}/ListadoTribunales_feather")
tribunales = df_tribunales['TRIBUNAL'].unique()

# DATAS INGRESO Y TERMINO MATERIA

### DATA INGRESO X MATERA

In [4]:
df_ingresos_materia = pd.read_feather(f"{path_interim}/IngresosMateria_feather")
df_ingresos_materia.head()


,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,COD. MATERIA,MATERIA,FECHA INGRESO
0,0,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,100-2018,Ordinaria,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-02
1,1,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,100-2018,Ordinaria,10004,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2018-04-02
2,2,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,101-2018,Ordinaria,12149,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2018-04-02
3,3,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,10-2018,Ordinaria,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-01-10
4,4,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,102-2018,Ordinaria,812,RECEPTACION. ART. 456 BIS A.,2018-04-02


In [5]:
# Normalizo nombres

df_ingresos_materia['TRIBUNAL'] = df_ingresos_materia['TRIBUNAL'].progress_apply(cleandata.cambio_nombre_juzgados)

100%|██████████| 3213259/3213259 [00:19<00:00, 165158.97it/s]


In [6]:
# Verifico si coinciden los nombres e imprimo los que estan en conflicto

for tribunal in df_ingresos_materia['TRIBUNAL'].unique():
    if not tribunal in tribunales:
        print(tribunal)

### TERMINO X MATERIA

In [7]:
df_termino_materia = pd.read_feather(f"{path_interim}/TerminoMateria_feather")
df_termino_materia.head()

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,COD. MATERIA,MATERIA,FECHA INGRESO,FECHA TERMINO,MOTIVO TERMINO,DURACION CAUSA
0,0,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,100-2018,Ordinaria,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-02,2018-06-01,SENTENCIA,60
1,1,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,100-2018,Ordinaria,10004,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2018-04-02,2018-06-01,SENTENCIA,60
2,2,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,101-2018,Ordinaria,12149,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2018-04-02,2018-08-29,SENTENCIA,149
3,3,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,10-2018,Ordinaria,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-01-10,2018-03-14,SENTENCIA,63
4,4,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,102-2018,Ordinaria,812,RECEPTACION. ART. 456 BIS A.,2018-04-02,2018-05-29,SENTENCIA,57


In [8]:
# Normalizo nombres

df_termino_materia['TRIBUNAL'] = df_termino_materia['TRIBUNAL'].progress_apply(cleandata.cambio_nombre_juzgados)

100%|██████████| 3606632/3606632 [00:21<00:00, 166888.23it/s]


In [9]:
# Verifico si coinciden los nombres e imprimo los que estan en conflicto

for tribunal in df_termino_materia['TRIBUNAL'].unique():
    if not tribunal in tribunales:
        print(tribunal)

## Data relacionada a Delitos

In [10]:
df_delitos = pd.read_feather(f"{path_processed}/Delitos_feather")
df_delitos.head()

,index,COD. MATERIA,MATERIA,TIPOLOGIA MATERIA,VIGENCIA MATERIA
0,0,101,QUEBRANTAMIENTO. ART. 90,LIBRO I TITULO IV QUEBRANTAMIENTO DE SENTEN...,VIGENTE
1,1,202,SECUESTRO. ART. 141,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
2,2,203,SUSTRACCION DE MENORES. ART. 142,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
3,3,204,VIOLACION DE MORADA. ART.144,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
4,4,205,"APERTURA , REGISTRO O INTERCEPTACION DE CORRE...",LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE


In [11]:
codigos_cot = df_delitos['COD. MATERIA'].unique()
print(f"Existen {len(codigos_cot)} codigos de delitos")

Existen 541 codigos de delitos


## Delitos en INGRESOS MATERIA

In [12]:
codigos_delitos = df_ingresos_materia['COD. MATERIA'].unique()

In [13]:
len(codigos_delitos)

474

In [14]:
codigos_no_encontrados = []
for delito in codigos_delitos:
    if not delito in codigos_cot:
        codigos_no_encontrados.append(delito)

In [15]:
codigos_no_encontrados

[13001, -31075, 0, 4011, 4014, 4013, 4010]

In [16]:
codigos_sin_registro = df_ingresos_materia[df_ingresos_materia['COD. MATERIA'].isin(codigos_no_encontrados)]

codigos_sin_registro['MATERIA'].unique()

# Analizar caso a caso:

array(['LESIONES LEVES.', 'FALTAS AL REGIMEN PENITENCIARIO', '[NULL]',
       'MANEJO EN ESTADO DE EBRIEDAD (SOLO CRIMEN)',
       'ESTAFA (SOLO CRIMEN)', 'ABUSO SEXUAL (SOLO CRIMEN)',
       'VIOLACION (SOLO CRIMEN)'], dtype=object)

#### Caso Codigo 13001 -> LESIONES LEVES
Este codigo debe tratarse como 13036 identificado como: LESIONES LEVES 494 Nº 5 CÓDIGO PENAL

In [17]:
df_ingresos_materia['COD. MATERIA'] = df_ingresos_materia['COD. MATERIA'].replace(13001,13036)

#### Caso Codigo -31075

In [18]:
filtro_codigo_31075 = df_ingresos_materia[df_ingresos_materia['COD. MATERIA']==-31075]
filtro_codigo_31075

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,COD. MATERIA,MATERIA,FECHA INGRESO
856574,205398,35,C.A. DE RANCAGUA,1070,JUZGADO DE GARANTIA RANCAGUA,14692-2019,Ordinaria,-31075,FALTAS AL REGIMEN PENITENCIARIO,2019-12-26
856575,205399,35,C.A. DE RANCAGUA,1070,JUZGADO DE GARANTIA RANCAGUA,14693-2019,Ordinaria,-31075,FALTAS AL REGIMEN PENITENCIARIO,2019-12-26
856576,205400,35,C.A. DE RANCAGUA,1070,JUZGADO DE GARANTIA RANCAGUA,14694-2019,Ordinaria,-31075,FALTAS AL REGIMEN PENITENCIARIO,2019-12-26
1065748,414572,56,C.A. DE PUERTO MONTT,1090,JUZGADO DE GARANTIA PUERTO MONTT,12196-2019,Ordinaria,-31075,FALTAS AL REGIMEN PENITENCIARIO,2019-12-23
1065749,414573,56,C.A. DE PUERTO MONTT,1090,JUZGADO DE GARANTIA PUERTO MONTT,12197-2019,Ordinaria,-31075,FALTAS AL REGIMEN PENITENCIARIO,2019-12-23
1065752,414576,56,C.A. DE PUERTO MONTT,1090,JUZGADO DE GARANTIA PUERTO MONTT,12200-2019,Ordinaria,-31075,FALTAS AL REGIMEN PENITENCIARIO,2019-12-23
1065756,414580,56,C.A. DE PUERTO MONTT,1090,JUZGADO DE GARANTIA PUERTO MONTT,12202-2019,Ordinaria,-31075,FALTAS AL REGIMEN PENITENCIARIO,2019-12-23
1161441,510265,90,C.A. DE SANTIAGO,1226,SEPTIMO JUZGADO DE GARANTIA SANTIAGO,13218-2019,Ordinaria,-31075,FALTAS AL REGIMEN PENITENCIARIO,2019-07-31


#### No corresponde a delito sino a sanción interna de gendarmeria, por lo que se eliminarán


In [19]:
df_ingresos_materia.drop(filtro_codigo_31075.index, axis = 0, inplace = True)

### Caso Codigo 0

In [20]:
filtro_codigo_0 = df_ingresos_materia[df_ingresos_materia['COD. MATERIA']==0]
filtro_codigo_0

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,COD. MATERIA,MATERIA,FECHA INGRESO
1132795,481619,90,C.A. DE SANTIAGO,1223,CUARTO JUZGADO DE GARANTIA SANTIAGO,1870-2019,Ordinaria,0,[NULL],2019-03-04
1138136,486960,90,C.A. DE SANTIAGO,1223,CUARTO JUZGADO DE GARANTIA SANTIAGO,6225-2019,Ordinaria,0,[NULL],2019-07-05


#### valores Null

In [21]:
df_ingresos_materia.drop(filtro_codigo_0.index, axis = 0, inplace = True)

### Codigos 4011, 4014, 4013, 4010
codigos No vigentes, mal cargados en sistemas 

In [22]:
filtro_codigo_4011 = df_ingresos_materia[df_ingresos_materia['COD. MATERIA']==4011]
filtro_codigo_4014 = df_ingresos_materia[df_ingresos_materia['COD. MATERIA']==4014]
filtro_codigo_4013 = df_ingresos_materia[df_ingresos_materia['COD. MATERIA']==4013]
filtro_codigo_4010 = df_ingresos_materia[df_ingresos_materia['COD. MATERIA']==4010]

In [23]:
df_ingresos_materia.drop(filtro_codigo_4011.index, axis = 0, inplace = True)
df_ingresos_materia.drop(filtro_codigo_4014.index, axis = 0, inplace = True)
df_ingresos_materia.drop(filtro_codigo_4013.index, axis = 0, inplace = True)
df_ingresos_materia.drop(filtro_codigo_4010.index, axis = 0, inplace = True)

## DELITOS EN TERMINO MATERIA

In [24]:
codigos_delitos = df_termino_materia['COD. MATERIA'].unique()

In [25]:
len(codigos_delitos)

468

In [26]:
codigos_no_encontrados = []
for delito in codigos_delitos:
    if not delito in codigos_cot:
        codigos_no_encontrados.append(delito)

In [27]:
codigos_no_encontrados

[13001, 4014, 4013, 4011, -31075, 4010, 0, 4007]

In [28]:
codigos_sin_registro = df_termino_materia[df_termino_materia['COD. MATERIA'].isin(codigos_no_encontrados)]

codigos_sin_registro['MATERIA'].unique()

# Analizar caso a caso:

array(['LESIONES LEVES.', 'ESTAFA (SOLO CRIMEN)',
       'ABUSO SEXUAL (SOLO CRIMEN)',
       'MANEJO EN ESTADO DE EBRIEDAD (SOLO CRIMEN)',
       'FALTAS AL REGIMEN PENITENCIARIO', 'VIOLACION (SOLO CRIMEN)',
       'LESIONES LEVES', '(en blanco)', 'HURTO (SOLO CRIMEN)',
       'NO REGISTRA CODIGO DE MATERIA'], dtype=object)

## Caso 13001 -- > LESIONES LEVES

In [29]:
df_termino_materia['COD. MATERIA'] = df_termino_materia['COD. MATERIA'].replace(13001,13036)

### Casos 4014, 4013, 4011, -31075, 4010, 0, 4007

In [30]:
filtro_codigo_4011 = df_termino_materia[df_termino_materia['COD. MATERIA']==4011]
filtro_codigo_4014 = df_termino_materia[df_termino_materia['COD. MATERIA']==4014]
filtro_codigo_4013 = df_termino_materia[df_termino_materia['COD. MATERIA']==4013]
filtro_codigo_4010 = df_termino_materia[df_termino_materia['COD. MATERIA']==4010]
filtro_codigo_31075 = df_termino_materia[df_termino_materia['COD. MATERIA']==31075]
filtro_codigo_4007 = df_termino_materia[df_termino_materia['COD. MATERIA']==4007]
filtro_codigo_0 = df_termino_materia[df_termino_materia['COD. MATERIA']==0]

In [31]:
df_termino_materia.drop(filtro_codigo_4011.index, axis = 0, inplace = True)
df_termino_materia.drop(filtro_codigo_4014.index, axis = 0, inplace = True)
df_termino_materia.drop(filtro_codigo_4013.index, axis = 0, inplace = True)
df_termino_materia.drop(filtro_codigo_4010.index, axis = 0, inplace = True)
df_termino_materia.drop(filtro_codigo_31075.index, axis = 0, inplace = True)
df_termino_materia.drop(filtro_codigo_4007.index, axis = 0, inplace = True)
df_termino_materia.drop(filtro_codigo_0.index, axis = 0, inplace = True)

### CONSOLIDACION DE DATA UNICO

In [41]:
pd.set_option('display.max_columns', 100)

In [38]:
df_consolidado = pd.merge(df_ingresos_materia, df_termino_materia, how='outer', on=['COD. TRIBUNAL','RIT','COD. MATERIA'])

In [40]:
df_consolidado

,index_x,COD. CORTE_x,CORTE_x,COD. TRIBUNAL,TRIBUNAL_x,RIT,TIPO CAUSA_x,COD. MATERIA,MATERIA_x,FECHA INGRESO_x,index_y,COD. CORTE_y,CORTE_y,TRIBUNAL_y,TIPO CAUSA_y,MATERIA_y,FECHA INGRESO_y,FECHA TERMINO,MOTIVO TERMINO,DURACION CAUSA
0,0.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,100-2018,Ordinaria,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-02,0.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-02,2018-06-01,SENTENCIA,60.0
1,1.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,100-2018,Ordinaria,10004,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2018-04-02,1.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2018-04-02,2018-06-01,SENTENCIA,60.0
2,2.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,101-2018,Ordinaria,12149,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2018-04-02,2.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2018-04-02,2018-08-29,SENTENCIA,149.0
3,3.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,10-2018,Ordinaria,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-01-10,3.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-01-10,2018-03-14,SENTENCIA,63.0
4,4.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,102-2018,Ordinaria,812,RECEPTACION. ART. 456 BIS A.,2018-04-02,4.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,RECEPTACION. ART. 456 BIS A.,2018-04-02,2018-05-29,SENTENCIA,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4713803,NaN,NaN,NaN,1323,NaN,251-2015,NaN,10009,NaN,NaT,721448.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,POSESION O TENENCIA DE ARMAS PROHIBIDAS,2015-12-10,2016-01-30,SENTENCIA,51.0
4713804,NaN,NaN,NaN,1323,NaN,251-2015,NaN,10099,NaN,NaT,721449.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,OTROS DELITOS DE LA LEY DE CONTROL DE ARMAS (L...,2015-12-10,2016-01-30,SENTENCIA,51.0
4713805,NaN,NaN,NaN,1323,NaN,89-2012,NaN,14060,NaN,NaT,721666.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,OTROS DELITOS CONTRA LA LEY DEL TRANSITO.,2012-09-12,2016-01-27,SENTENCIA,1232.0
4713806,NaN,NaN,NaN,1323,NaN,91-2013,NaN,812,NaN,NaT,721669.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,RECEPTACION. ART. 456 BIS A.,2013-09-05,2016-09-12,SENTENCIA,1103.0


## CARGA DATA A ARCHIVOS FUTHER

In [147]:
# Directorio donde se guardaran archivos feather
df_ingresos_materia.reset_index(inplace = True)
df_termino_materia.reset_index(inplace = True)
df_consolidado.reset_index(inplace = True)
os.makedirs(path_processed, exist_ok = True)  

# Guardamos dataset como archivo feather

df_ingresos_materia.to_feather(f'{path_processed}/IngresosMateria_feather')
df_termino_materia.to_feather(f'{path_processed}/TerminoMateria_feather')
df_consolidado.to_feather(f'{path_processed}/Consolidado_Materia_feather')

ValueError: Duplicate column names found: ['level_0', 'index', 'COD. CORTE', 'CORTE', 'COD. TRIBUNAL', 'TRIBUNAL', 'RIT', 'TIPO CAUSA', 'COD. MATERIA', 'MATERIA', 'FECHA INGRESO', 'TEMP', 'index', 'COD. CORTE', 'CORTE', 'COD. TRIBUNAL', 'TRIBUNAL', 'RIT', 'TIPO CAUSA', 'COD. MATERIA', 'MATERIA', 'FECHA INGRESO', 'FECHA TERMINO', 'MOTIVO TERMINO', 'DURACION CAUSA', 'TEMP']